In [8]:
import pandas as pd

custom_alert_rule_path = r"C:\Users\revital.tomchin\Downloads\data files bi developer hw\custom_alert_rule.csv"
disconnection_alert_rule_results_path = r"C:\Users\revital.tomchin\Downloads\data files bi developer hw\disconnection_alert_rule_results.csv"
alert_record_results_path = r"C:\Users\revital.tomchin\Downloads\data files bi developer hw\alert_record_results.csv"

custom_alert_rule = pd.read_csv(custom_alert_rule_path)
disconnection_alert_rule_results = pd.read_csv(disconnection_alert_rule_results_path)
alert_record_results = pd.read_csv(alert_record_results_path)

# data formats
def standardize_dates(df, date_columns):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')  # Coerce invalid dates to NaT
    return df

date_columns_alert_rule = ['createdDate', 'lastUpdated']
date_columns_alert_record = ['createdDate', 'updatedDate']

custom_alert_rule = standardize_dates(custom_alert_rule, date_columns_alert_rule)
disconnection_alert_rule_results = standardize_dates(disconnection_alert_rule_results, date_columns_alert_rule)
alert_record_results = standardize_dates(alert_record_results, date_columns_alert_record)

# initial dupe removal
custom_alert_rule.drop_duplicates(subset='id', inplace=True)
disconnection_alert_rule_results.drop_duplicates(subset='id', inplace=True)
alert_record_results.drop_duplicates(subset='id', inplace=True)

# standartizing column names
def normalize_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.strip()
    return df

custom_alert_rule = normalize_column_names(custom_alert_rule)
disconnection_alert_rule_results = normalize_column_names(disconnection_alert_rule_results)
alert_record_results = normalize_column_names(alert_record_results)

# set category data
def convert_to_category(df, columns):
    for col in columns:
        df[col] = df[col].astype('category')
    return df

categorical_columns_alert_rule = ['state']
categorical_columns_alert_record = ['acknowledged', 'inmaintenance']

custom_alert_rule = convert_to_category(custom_alert_rule, categorical_columns_alert_rule)
disconnection_alert_rule_results = convert_to_category(disconnection_alert_rule_results, categorical_columns_alert_rule)
alert_record_results = convert_to_category(alert_record_results, categorical_columns_alert_record)

print("Data cleaning steps are complete.")


Data cleaning steps are complete.


Cleaning custom rule

In [10]:
def extract_duration_units_if_exists(df, column_name):
    if 'duration' not in df.columns or 'units' not in df.columns:
        df['duration'] = df.get('duration', None)
        df['units'] = df.get('units', None)

    def extract_values(value):
        if pd.isna(value):
            return None, None
        try:
            parsed_value = json.loads(value)
            duration = parsed_value.get('duration')
            units = parsed_value.get('units')
            return duration, units
        except (json.JSONDecodeError, TypeError):
            return None, None

    df['duration'], df['units'] = zip(*df[column_name].apply(extract_values))

    return df


,id,alerttype,status,conditionsvalues,conditionsvalueshistory,createddate,updateddate,customalertruleid,manufactureralertruleid,acknowledged,disconnectionalertruleid,genericalertruleid,inmaintenance
0,0f457c23-d390-4c4f-a293-8998b8f37b79,deviceDisconnection,close,{Connected},"[{""timeOfUpdate"": ""2023-12-11T23:59:38.180Z"", ...",2023-12-11 23:59:38.180,2023-12-12 00:04:54.755,NaN,NaN,True,4729d76b-ccf8-461e-915c-b8bd3258d857,NaN,False
1,e597c032-da09-4875-b311-2164fbb674df,custom,close,NaN,"[{""timeOfUpdate"": ""2023-08-27T16:27:00.000Z""},...",NaT,NaT,6bcaf071-00f6-40a8-a0e9-808b3240ca91,NaN,True,NaN,NaN,False
2,ebdf7013-313b-43bd-a0a4-c7d98ef25234,custom,close,NaN,"[{""timeOfUpdate"": ""2023-08-28T10:56:00.000Z""},...",NaT,NaT,6c92bfad-83f9-4902-98e9-74b2f21407e1,NaN,True,NaN,NaN,False
3,702ae016-7b9d-4fc4-b48b-538b814132ff,custom,close,NaN,"[{""timeOfUpdate"": ""2023-12-03T12:29:00.000Z""},...",NaT,NaT,f2cb88dc-95c2-48c9-ac0d-59b81eba0e55,NaN,True,NaN,NaN,False
4,d3178e55-c5d0-453d-833e-e33578b67f88,custom,close,NaN,"[{""timeOfUpdate"": ""2023-12-03T12:29:00.000Z""},...",NaT,NaT,6886384f-14a4-4da7-9d99-5b904e3886b1,NaN,True,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7383,cebcdafb-6b4f-4900-a72d-fa32aa8e7bda,custom,close,NaN,"[{""timeOfUpdate"": ""2024-09-11T05:04:00.000Z""},...",NaT,NaT,37a3b71b-ba20-45ce-92cc-4e76a69b53a8,NaN,False,NaN,NaN,False
7384,170351b5-9953-428c-ba5d-1040734bbe6a,custom,close,NaN,"[{""timeOfUpdate"": ""2024-09-11T13:16:00.000Z""},...",NaT,NaT,bc27a4a5-b9c9-409e-a050-af90e9df0ec4,NaN,False,NaN,NaN,False
7385,aab25730-6610-425a-99fd-9980e54654f6,custom,close,NaN,"[{""timeOfUpdate"": ""2024-09-12T08:40:00.000Z""},...",NaT,NaT,bc27a4a5-b9c9-409e-a050-af90e9df0ec4,NaN,False,NaN,NaN,False
7386,4f82005b-3047-47b5-9e6d-0ea5efc20f08,custom,close,NaN,"[{""timeOfUpdate"": ""2024-09-12T04:05:00.000Z""},...",NaT,NaT,37a3b71b-ba20-45ce-92cc-4e76a69b53a8,NaN,False,NaN,NaN,False
